In [1]:
import math
import time
import random 
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors
from statistics import stdev

In [2]:
def partialcorr(X,Y,Z):
    xy = np.corrcoef(X,Y)[0,1]
    xz = np.corrcoef(X,Z)[0,1]
    yz = np.corrcoef(Y,Z)[0,1]
    
    return (xy-xz*yz)/(math.sqrt(1-xz**2)*math.sqrt(1-yz**2))

# データ作成

In [3]:
def make_data(L,m,byx,byz,bzx):
    x = np.empty((m,L))
    y = np.empty((m,L))
    z = np.empty((m,L))
    
    #初期値
    x0=np.linspace(0.1,0.8,m)
    y0=np.linspace(0.1,0.8,m)
    z0=np.linspace(0.1,0.8,m)

    for i in range(m):
        x[i,0]=x0[i]
        y[i,0]=y0[i]
        z[i,0]=z0[i]
    
    #copled logistic式
    for i in range(m):
        for j in range(L-1):
            x[i,j+1]=x[i,j]*(rx-rx*x[i,j])#+np.random.normal(0,0.005)
            y[i,j+1]=y[i,j]*(ry-ry*y[i,j]-byx*x[i,j]-byz*z[i,j])
            z[i,j+1]=z[i,j]*(rz-rz*z[i,j]-bzx*x[i,j])#+np.random.normal(0,0.005)

    return x,y,z

# Simplex projection(Simplex法)

In [4]:
def simplex(x,y,z,maxE,L,m):
    #相関係数を格納
    cX=[]
    cY=[]
    cZ=[]

    #各埋め込み次元で計算
    for a in range(1,maxE):
        E=a+1
        
        #正解データ
        x1=x[:,E:]
        y1=y[:,E:]
        z1=z[:,E:]
        X_=np.reshape(x1,np.size(x1))
        Y_=np.reshape(y1,np.size(y1))
        Z_=np.reshape(z1,np.size(z1))
        
        #予測データ
        X=[]
        Y=[]
        Z=[]
        
        #時間遅れベクトルの作成
        for i in range(m):
            for j in range(E-1,L-1):
                x1=[]
                y1=[]
                z1=[]
                for k in range(E):
                    x1.append(x[i,j-k])
                    y1.append(y[i,j-k])
                    z1.append(z[i,j-k])
                X.append(x1)
                Y.append(y1)
                Z.append(z1)
        X=np.reshape(X,(int(np.size(X)/(E)),E))
        Y=np.reshape(Y,(int(np.size(Y)/(E)),E))
        Z=np.reshape(Z,(int(np.size(Z)/(E)),E))
        
        #予測 
        nei=E+1
        neighX = NearestNeighbors(n_neighbors=nei+1, algorithm='ball_tree').fit(X)
        neighY = NearestNeighbors(n_neighbors=nei+1, algorithm='ball_tree').fit(Y)
        neighZ = NearestNeighbors(n_neighbors=nei+1, algorithm='ball_tree').fit(Z)
        X1=[]
        Y1=[]
        Z1=[]
        for i in range(len(X)):
            disX, indX = neighX.kneighbors([X[i]])
            disY, indY = neighY.kneighbors([Y[i]])
            disZ, indZ = neighZ.kneighbors([Z[i]])
            disX=disX[0]
            disY=disY[0]
            disZ=disZ[0]
            indX=indX[0]
            indY=indY[0]
            indZ=indZ[0]

            UX=np.zeros(nei)
            UY=np.zeros(nei)
            UZ=np.zeros(nei)
            sumUX=0
            sumUY=0
            sumUZ=0
            
            #距離が0だとエラーが発生するのでその対処
            for j in range(nei):
                if disX[j+1]==0:
                    disX[j+1]=0.0001
                if disY[j+1]==0:
                    disY[j+1]=0.0001
                if disZ[j+1]==0:
                    disZ[j+1]=0.0001
                #重みの計算
                UX[j]=np.exp(-disX[j+1]/disX[1])
                UY[j]=np.exp(-disY[j+1]/disY[1])
                UZ[j]=np.exp(-disZ[j+1]/disZ[1])
                sumUX+=UX[j]
                sumUY+=UY[j]
                sumUZ+=UZ[j]
            sumYX=0
            sumYY=0
            sumYZ=0
            #予測座標の計算
            for j in range(nei):
                sumYX+=(UX[j]/sumUX)*X_[indX[j+1]]
                sumYY+=(UY[j]/sumUY)*Y_[indY[j+1]]
                sumYZ+=(UZ[j]/sumUZ)*Z_[indZ[j+1]]
            X1.append(sumYX)
            Y1.append(sumYY)
            Z1.append(sumYZ)
        
        #相関係数を計算
        cX.append(np.corrcoef(X1,X_)[0,1])
        cY.append(np.corrcoef(Y1,Y_)[0,1])
        cZ.append(np.corrcoef(Z1,Z_)[0,1])
    
    #相関係数が最大のものを選択
    E_X=int(np.argmax(cX)+2)
    E_Y=int(np.argmax(cY)+2)
    E_Z=int(np.argmax(cZ)+2)
    E=E_X,E_Y,E_Z

    return E_X,E_Y,E_Z

# bootstrapping

In [5]:
def boot(x,y,z,E,tau,iteration=100):
    #１次元配列に変換
    A=np.reshape(x,np.size(x))
    B=np.reshape(y,np.size(y))
    C=np.reshape(z,np.size(z))
    
    #正解データのインデックスを計算
    a=np.arange(m*(L))  #!is.nan
    a=np.reshape(a,(m,L))
    a=a[:,tau*(E-1):]
    a=np.reshape(a,np.size(a))
    acceptablelib=[i for i in a if i<np.size(x)]
    acceptablelib=np.reshape(acceptablelib,np.size(acceptablelib))
    a=np.reshape(a,np.size(a))

    length=np.size(x)  #時系列長
    #予測時系列
    Aest=np.zeros(len(A))  
    Cest=np.zeros(len(A))
    AAest=np.zeros(len(A))
    rho=np.zeros(len(A))  #相関係数
    rhoD=np.zeros(len(A))  #偏相関係数
    
    X=[]
    Y=[]
    #時間遅れベクトルを作成
    for i in acceptablelib:
        X_e=[]
        Y_e=[]
        for j in range(E):
            X_e.append(A[i-tau*j])
            Y_e.append(B[i-tau*j])            
        X.append(X_e)
        Y.append(Y_e)
    X=np.reshape(X,(int(np.size(X)/(E)),E))
    Y=np.reshape(Y,(int(np.size(Y)/(E)),E))
    
    nei=E+1  #近傍点の数
    
    #ランダムに選択する座標の数を計算
    acceptablelib2=acceptablelib[acceptablelib<((length-1)-(tau))]
    DesiredL=np.arange(tau*(E-1)+(E+1),len(A)-(E-2)+1)
    DesiredL+=E-2+1
    for i in range(len(DesiredL)):
        DesiredL[i]=acceptablelib2[np.argmin(abs(acceptablelib2-DesiredL[i]))]
    DesiredL=np.unique(DesiredL)

    lenacceptablelib=len(acceptablelib)

    Aest_mat=[]  #すべてのAestを格納
    rho_mat=[]  #すべてのrhoを格納
    rhoD_mat=[]  #すべてのrhoDを格納

    DesiredL=[DesiredL[0],DesiredL[-1]]  #計算時間短縮用

    #bootstrapping
    for itlst in range(iteration):
        random.seed(itlst)
        
        from_=tau*(E-1)  #スタート地点
        #ランダムに選択する座標のインデックス
        LibUse=np.zeros(length)
        LibUse=LibUse.astype(int)
        #近傍点のインデックス
        neighbors=np.zeros(E+1)
        neighbors=neighbors.astype(int)
        lpos=[]
        
        #選択する座標の数を増やしながら予測
        for l in DesiredL:
            if(l<(from_+E+1)):
                l=from_+E+1
            elif(l>=length):
                l=length-1
            to=l
            
            #座標をランダムに選択
            Y_rand=[]
            Y_rand_ind=[]
            for i in range(l):
                count=random.randint(0,lenacceptablelib-1)
                Y_rand.append(Y[count])
                Y_rand_ind.append(count)
            Y_rand=np.reshape(Y_rand,(int(np.size(Y_rand)/E),E))
            
            neigh = NearestNeighbors(n_neighbors=nei+2, algorithm='ball_tree').fit(Y_rand[:l])
    
            #選択した座標から選択
            for i in range(lenacceptablelib):
                ac=acceptablelib[i]
                
                dis,ind=neigh.kneighbors([Y[i]]) 
                if dis[0,0]==0:
                    dis=dis[0,1:]
                    ind=ind[0,1:]
                else:
                    dis=dis[0]
                    ind=ind[0]

                distsv=dis[0]  #最近傍点の距離
                sumaest=0
                sumu=0
                sumw=0
                w=np.zeros(E+1)
                
                #最近傍点が0の場合
                if distsv!=0:
                    u=np.zeros(E+1)
                    for j in range(nei):
                        u[j]=np.exp(-dis[j]/distsv)
                    w=u/sum(u)                
                    for j in range(nei):
                        if w[j]<0.0001:
                            w[j]=0.0001
                    w=w/sum(w) 
                    for j in range(nei):
                        sumaest+=A[a[Y_rand_ind[ind[j]]]]*w[j]
                else:
                    for j in range(nei):
                        if dis[j]==0:
                            w[j]=1
                        else:
                            w[j]=0.0001
                    w=w/sum(w)
                    
                    for j in range(nei):
                        sumaest+=A[a[Y_rand_ind[ind[j+1]]]]*w[j]
                Aest[ac]=sumaest
            
            #間接時系列の予測 y->z
            for i in range(lenacceptablelib):
                ac=acceptablelib[i]
                dis,ind=neigh.kneighbors([Y[i]]) 
                if dis[0,0]==0:
                    dis=dis[0,1:]
                    ind=ind[0,1:]
                else:
                    dis=dis[0]
                    ind=ind[0]
                
                distsv=dis[0]
                sumaest=0
                sumu=0
                sumw=0
                w=np.zeros(E+1)
                if distsv!=0:
                    u=np.zeros(E+1)
                    for j in range(nei):
                        u[j]=np.exp(-dis[j]/distsv)
                    w=u/sum(u)                
                    for j in range(nei):
                        if w[j]<0.0001:
                            w[j]=0.0001
                    w=w/sum(w) 
                    for j in range(nei):
                        sumaest+=C[a[Y_rand_ind[ind[j]]]]*w[j]
                else:
                    for j in range(nei):
                        if dis[j]==0:
                            w[j]=1
                        else:
                            w[j]=0.0001
                    w=w/sum(w)
                    for j in range(nei):
                        sumaest+=C[a[Y_rand_ind[ind[j+1]]]]*w[j]
                Cest[ac]=sumaest
            
            #間接時系列の予測 z->x
            Z=[]
            for i in acceptablelib:
                Z_e=[]
                for j in range(E):
                    Z_e.append(Cest[i-tau*j])       
                Z.append(Z_e)
            Z=np.reshape(Z,(int(np.size(Z)/(E)),E))
            neigh=NearestNeighbors(n_neighbors=nei+2, algorithm='ball_tree').fit(Z[:l])
            for i in range(lenacceptablelib):
                ac=acceptablelib[i]
                dis,ind=neigh.kneighbors([Z[i]]) 
                if dis[0,0]==0:
                    dis=dis[0,1:]
                    ind=ind[0,1:]
                else:
                    dis=dis[0]
                    ind=ind[0]
                
                distsv=dis[0]
                sumaest=0
                sumu=0
                sumw=0
                w=np.zeros(E+1)
                if distsv!=0:
                    u=np.zeros(E+1)
                    for j in range(nei):
                        u[j]=np.exp(-dis[j]/distsv)
                    w=u/sum(u)                
                    for j in range(nei):
                        if w[j]<0.0001:
                            w[j]=0.0001
                    w=w/sum(w) 
                    for j in range(nei):
                        sumaest+=A[a[ind[j]]]*w[j]
                else:
                    for j in range(nei):
                        if dis[j+1]==0:
                            w[j]=1
                        else:
                            w[j]=0.0001
                    w=w/sum(w)
                    for j in range(nei):
                        sumaest+=A[a[ind[j+1]]]*w[j]
                AAest[ac]=sumaest
            
            #相関、偏相関の計算
            rho[l]=np.corrcoef(A[acceptablelib],Aest[acceptablelib])[1,0]
            rhoD[l]=partialcorr(A[acceptablelib],Aest[acceptablelib],AAest[acceptablelib])
            
            lpos.append(l)

        #copyしないと参照渡しになるみたい
        Aest_mat.append(Aest.copy())
        rho_mat.append(rho.copy())
        rhoD_mat.append(rhoD.copy())
    
    #配列を整形
    Aest_mat=np.reshape(Aest_mat,(iteration,int(np.size(Aest_mat)/iteration))).T
    rho_mat=np.reshape(rho_mat,(iteration,int(np.size(rho_mat)/iteration))).T
    rhoD_mat=np.reshape(rhoD_mat,(iteration,int(np.size(rhoD_mat)/iteration))).T
    
    #平均を求める
    e=[np.mean(Aest_mat[i]) for i in lpos]
    r=[np.mean(rho_mat[i]) for i in lpos]
    rD=[np.mean(rhoD_mat[i]) for i in lpos]

    #上回る回数を数える
    CCM_boot=(sum(rho_mat[lpos[0]]<rho_mat[lpos[-1]])/iteration)
    PCM_boot=(sum(rhoD_mat[lpos[0]]<rhoD_mat[lpos[-1]])/iteration)
    
    return np.array(e),np.array(r),np.array(rD),CCM_boot,PCM_boot,np.array(DesiredL)

In [6]:
def main(L,m,byx,byz,bzx,iteration):
    CCM_boot_yx=[]
    CCM_boot_xy=[]
    PCM_boot_yx=[]
    PCM_boot_xy=[]

    for i in range(n):
        #yx x->y, xy y->x
        x,y,z=make_data(L,m,byx[i],byz[i],bzx[i])
        E_X,E_Y,E_Z=simplex(x,y,z,maxE,L,m)
        #x->y
        e,r,eD,CCM_boot,PCM_boot,lpos=boot(x,y,z,E_Y,tau,iteration)
        CCM_boot_yx.append(CCM_boot)
        PCM_boot_yx.append(PCM_boot)
        #y->x
        e,r,eD,CCM_boot,PCM_boot,lpos=boot(y,x,z,E_Y,tau,iteration)
        CCM_boot_xy.append(CCM_boot)
        PCM_boot_xy.append(PCM_boot)
        
        print(i)
    return CCM_boot_yx,CCM_boot_xy,PCM_boot_yx,PCM_boot_xy

In [7]:
rx=3.6  #xの自身への影響度
ry=3.72  #yの自身への影響度
rz=3.68  #zの自身への影響度
n=100  #繰り返す回数
maxE=5  #最大の埋め込み次元
tau=1  #時間遅れ
iteration=100  #繰り返し回数
rate=0.95  #割合

In [8]:
byx=np.zeros(n)  #xがyからの影響度

random.seed(100)
byz=[random.uniform(0.2,0.9) for i in range(n)]  #yがxからの影響度
bzx=[random.uniform(0.2,0.9) for i in range(n)]
# print(byx)

In [ ]:
L=20
m=5
t=time.time()
CCM_yx_5,CCM_xy_5,PCM_yx_5,PCM_xy_5=main(L,m,byx,byz,bzx,iteration)
print(time.time()-t)

In [ ]:
C=[1 if rate<=i else 0 for i in CCM_yx_5]
print(sum(C)/n)
c=[1 if 0.95<=i else 0 for i in CCM_xy_5]
print(sum(c)/n)
P=[1 if 0.95<=i else 0 for i in PCM_yx_5]
print(sum(P)/n)
p=[1 if 0.95<=i else 0 for i in PCM_xy_5]
print(sum(p)/n)

correct=0
for i in range(n):
    if C[i]==1 and c[i]==0 and P[i]==0 and p[i]==0:
        correct+=1
print("正解率:",correct)

In [ ]:
L=20
m=10
t=time.time()
CCM_yx_10,CCM_xy_10,PCM_yx_10,PCM_xy_10=main(L,m,byx,byz,bzx,iteration)
print(time.time()-t)

In [ ]:
C=[1 if 0.95<=i else 0 for i in CCM_yx_10]
print(sum(C)/n)
c=[1 if 0.95<=i else 0 for i in CCM_xy_10]
print(sum(c)/n)
P=[1 if 0.95<=i else 0 for i in PCM_yx_10]
print(sum(P)/n)
p=[1 if 0.95<=i else 0 for i in PCM_xy_10]
print(sum(p)/n)

correct=0
for i in range(n):
    if C[i]==1 and c[i]==0 and P[i]==0 and p[i]==0:
        correct+=1
print("正解率:",correct)

In [ ]:
L=20
m=15
t=time.time()
CCM_yx_15,CCM_xy_15,yx_sd_15,xy_sd_15,PCM_yx_15,PCM_xy_15,yx_sdd_15,xy_sdd_15=main(L,m,byx,byz,bzx,iteration)
print(time.time()-t)

In [ ]:
C=[1 if rate<=i else 0 for i in CCM_yx_15]
print(sum(C)/n)
c=[1 if rate<=i else 0 for i in CCM_xy_15]
print(sum(c)/n)
P=[1 if rate<=i else 0 for i in PCM_yx_15]
print(sum(P)/n)
p=[1 if rate<=i else 0 for i in PCM_xy_15]
print(sum(p)/n)

correct=0
for i in range(n):
    if C[i]==1 and c[i]==0 and P[i]==0 and p[i]==0:
        correct+=1
        tr.append([byz[i],bzx[i]])
    else:
        fa.append([byz[i],bzx[i]])
print("正解率:",correct)

In [ ]:
L=20
m=20
t=time.time()
CCM_yx_20,CCM_xy_20,yx_sd_20,xy_sd_20,PCM_yx_20,PCM_xy_20,yx_sdd_20,xy_sdd_20=main(L,m,byx,byz,bzx,iteration)
print(time.time()-t)

In [ ]:
C=[1 if rate<=i else 0 for i in CCM_yx_20]
print(sum(C)/n)
c=[1 if rate<=i else 0 for i in CCM_xy_20]
print(sum(c)/n)
P=[1 if rate<=i else 0 for i in PCM_yx_20]
print(sum(P)/n)
p=[1 if rate<=i else 0 for i in PCM_xy_20]
print(sum(p)/n)

correct=0
for i in range(n):
    if C[i]==1 and c[i]==0 and P[i]==0 and p[i]==0:
        correct+=1
print("正解率:",correct)

In [ ]:
L=20
m=25
t=time.time()
CCM_yx_25,CCM_xy_25,yx_sd_25,xy_sd_25,PCM_yx_25,PCM_xy_25,yx_sdd_25,xy_sdd_25=main(L,m,byx,byz,bzx,iteration)
print(time.time()-t)

In [ ]:
C=[1 if rate<=i else 0 for i in CCM_yx_25]
print(sum(C)/n)
c=[1 if rate<=i else 0 for i in CCM_xy_25]
print(sum(c)/n)
P=[1 if rate<=i else 0 for i in PCM_yx_25]
print(sum(P)/n)
p=[1 if rate<=i else 0 for i in PCM_xy_25]
print(sum(p)/n)

correct=0
for i in range(n):
    if C[i]==1 and c[i]==0 and P[i]==0 and p[i]==0:
        correct+=1
print("正解率:",correct)